In [1]:
pip install torchdiffeq

Note: you may need to restart the kernel to use updated packages.


In [1]:
#############################

import torchdiffeq

import torch
import torchdiffeq
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing

# Define system parameters
m = 6e4  # mass (kg)
k = 5e6  # 
f = 0.05  # damping ratio
T = 0.69  # natural period (s)
c = 2 * m * f * np.sqrt(k/m)  # damping coefficixent
a = 0.1  # hysteresis parameter
uy = 0.04  # yielding displacement (m)
duration = 8.0  # duration for response calculation (s)

# Bouc-Wen model parameters
A = 1
n = 3  # power for Bouc-Wen
gamma = 1 / (2 * (uy**n))
beta = 1 / (2 * (uy**n))

# Ground acceleration parameters (white noise)
S = 0.005 #0.005
n_freq = 300  # number of frequency points
cutoff_freq = 15 * np.pi  # cutoff frequency (rad/s)
dw = 30 * np.pi / n_freq  # frequency step


class HystereticOscillator(torch.nn.Module):
    def __init__(self, m, c, k, a, A, gamma, beta, n, u_j, u_hat_j):
        super(HystereticOscillator, self).__init__()
        self.m = m
        self.c = c
        self.k = k
        self.a = a
        self.A = A
        self.gamma = gamma
        self.beta = beta
        self.n = n
        self.u_j = u_j
        self.u_hat_j = u_hat_j
        self.time = torch.linspace(0, duration, 400)

        # Convert S and dw to tensors
        self.S = torch.tensor(S, dtype=torch.float32)
        self.dw = torch.tensor(dw, dtype=torch.float32)

    def forward(self, t, state):
        X, V, Z = state
        
        # Ground acceleration
        omega_j = torch.arange(1, (n_freq // 2)+1) * self.dw  # Shape: (n_freq // 2 - 1,)
        sqrt_term = torch.sqrt(2 * self.S * self.dw)  # Scalar

        # Broadcast omega_j with t for cosine and sine terms
        cos_term = torch.cos(omega_j* t)  # Shape: (n_freq // 2 - 1, len(t))
        sin_term = torch.sin(omega_j* t)  # Shape: (n_freq // 2 - 1, len(t))


        # Compute ground_acc
        ground_acc = sqrt_term * (self.u_j * cos_term +
                                  self.u_hat_j * sin_term)

        # ground_acc = ground_acc.sum(dim=0)
        ground_acc = torch.sum(ground_acc)

        
        # Dynamics of the oscillator
        acc = ((-ground_acc * self.m) - (self.c * V) - self.k * (self.a * X + (1 - self.a) * Z)) / self.m
        dZ = (-self.gamma * torch.abs(V) * (torch.abs(Z)**(self.n-1)) * Z - self.beta * (torch.abs(Z)**self.n) * V + self.A * V)
        
        return torch.stack([V, acc, dZ])


# Single simulation function
def run_simulation(x_threshold):
    # Generate random variables for ground motion
    u_j = torch.randn(n_freq // 2)
    u_hat_j = torch.randn(n_freq // 2)

    # Initialize the oscillator
    oscillator = HystereticOscillator(m, c, k, a, A, gamma, beta, n, u_j, u_hat_j)

    # Initial state: [displacement, velocity, hysteresis]
    initial_state = torch.tensor([0.0, 0.0, 0.0])

    # Solve the ODE using torchdiffeq and RK4
    solution = torchdiffeq.odeint(oscillator, initial_state, oscillator.time, method='rk4')

    # Extract the displacement (X) at the last time step
    final_displacement = solution[-1, 0].item()

    # Check if failure occurs
    return final_displacement >= x_threshold

n_iter = 5*(1e4)
fcount = 0; 
for i in range( int(n_iter) ):
    if run_simulation(uy):
        fcount += 1
    if(i%1 == 0):
      print("nfailures :", fcount, " niter:", i+1, "current prob ", fcount/(i+1))

print("prob: ", fcount/n_iter)

nfailures : 0  niter: 1 current prob  0.0
nfailures : 0  niter: 2 current prob  0.0
nfailures : 0  niter: 3 current prob  0.0
nfailures : 0  niter: 4 current prob  0.0
nfailures : 0  niter: 5 current prob  0.0
nfailures : 0  niter: 6 current prob  0.0
nfailures : 0  niter: 7 current prob  0.0
nfailures : 0  niter: 8 current prob  0.0
nfailures : 0  niter: 9 current prob  0.0
nfailures : 0  niter: 10 current prob  0.0
nfailures : 0  niter: 11 current prob  0.0
nfailures : 0  niter: 12 current prob  0.0
nfailures : 0  niter: 13 current prob  0.0
nfailures : 0  niter: 14 current prob  0.0
nfailures : 0  niter: 15 current prob  0.0
nfailures : 0  niter: 16 current prob  0.0
nfailures : 0  niter: 17 current prob  0.0
nfailures : 0  niter: 18 current prob  0.0
nfailures : 0  niter: 19 current prob  0.0
nfailures : 0  niter: 20 current prob  0.0
nfailures : 0  niter: 21 current prob  0.0
nfailures : 0  niter: 22 current prob  0.0
nfailures : 0  niter: 23 current prob  0.0
nfailures : 0  niter

KeyboardInterrupt: 